# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [41]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,el cope,8.6167,-80.6000,26.05,61,89,3.08,PA,1682460877
1,1,port alfred,-33.5906,26.8910,20.10,97,99,3.89,ZA,1682460866
2,2,jamestown,42.0970,-79.2353,8.07,68,75,4.63,US,1682460812
3,3,castro,-24.7911,-50.0119,15.61,78,92,1.43,BR,1682461091
4,4,roald,62.5833,6.1333,2.73,93,75,3.60,NO,1682460878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
city_map_plot= city_data_df.hvplot.points(
    'Lng',
    'Lat', 
    geo= True, 
    tiles= 'OSM',
    size= 'Humidity',
    color='City',
    frame_width= 700,
    frame_height= 500
)

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
cities_plot_df= city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] <= 10)]

# Drop any rows with null values
cities_plot_df = cities_plot_df.dropna(how= 'any')

# Display sample data
cities_plot_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,sao filipe,14.8961,-24.4956,23.55,72,6,3.39,CV,1682460883
56,56,baghdad,33.3406,44.4009,22.95,35,0,2.57,IQ,1682460798
61,61,qarqin,37.4185,66.0436,22.96,26,0,1.35,AF,1682460888
64,64,sao joao da barra,-21.6403,-41.0511,24.05,86,4,3.43,BR,1682460803
75,75,saint george,37.1041,-113.5841,22.92,17,0,4.47,US,1682460778
121,121,souillac,-20.5167,57.5167,25.00,84,9,1.24,MU,1682460897
162,162,sohag,26.5570,31.6948,23.68,21,0,2.91,EG,1682460906
254,254,kishanganj,26.1167,87.9333,24.76,43,1,2.13,IN,1682460924
288,288,rawah,34.4816,41.9093,22.34,27,1,3.06,IQ,1682460931
290,290,dawlatabad,36.4141,64.9053,21.93,21,0,1.62,AF,1682460932


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= cities_plot_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", " ")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
32,sao filipe,CV,14.8961,-24.4956,72,
56,baghdad,IQ,33.3406,44.4009,35,
61,qarqin,AF,37.4185,66.0436,26,
64,sao joao da barra,BR,-21.6403,-41.0511,86,
75,saint george,US,37.1041,-113.5841,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
limit=20
params = {
    "limit": limit,
    "type":"accomodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df["Lng"]
    latitude = hotel_df["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:" + str(longitude[index]) + "," + str(latitude[index]) + "," + str(radius)
    params["bias"] = "proximity:" + str(longitude[index]) + "," + str(latitude[index])
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sao filipe - nearest hotel: No hotel found
baghdad - nearest hotel: No hotel found
qarqin - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
saint george - nearest hotel: No hotel found
souillac - nearest hotel: No hotel found
sohag - nearest hotel: No hotel found
kishanganj - nearest hotel: No hotel found
rawah - nearest hotel: No hotel found
dawlatabad - nearest hotel: No hotel found
doha - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
tambopata - nearest hotel: No hotel found
panan - nearest hotel: No hotel found
ondjiva - nearest hotel: No hotel found
olhao - nearest hotel: No hotel found
high rock - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
32,sao filipe,CV,14.8961,-24.4956,72,No hotel found
56,baghdad,IQ,33.3406,44.4009,35,No hotel found
61,qarqin,AF,37.4185,66.0436,26,No hotel found
64,sao joao da barra,BR,-21.6403,-41.0511,86,No hotel found
75,saint george,US,37.1041,-113.5841,17,No hotel found
121,souillac,MU,-20.5167,57.5167,84,No hotel found
162,sohag,EG,26.5570,31.6948,21,No hotel found
254,kishanganj,IN,26.1167,87.9333,43,No hotel found
288,rawah,IQ,34.4816,41.9093,27,No hotel found
290,dawlatabad,AF,36.4141,64.9053,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
cities_plot_df= hotel_df.hvplot.points(
    'Lng',
    'Lat', 
    geo= True, 
    tiles= 'OSM',
    size= 'Humidity',
    color='City',
    frame_width= 700,
    frame_height= 500
)

# Display the map
cities_plot_df

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)